## 5 Blast

O Biopython também tem um pacote específico para fazer comparação de sequências e aqui vamos dar uma olhada no programa Blast. Para rodar Blast Web pela API do NCBI devemos importar o NCBIWWW do pacote Blast.

In [38]:
from Bio.Blast import NCBIWWW
help(NCBIWWW)

Help on module Bio.Blast.NCBIWWW in Bio.Blast:

NAME
    Bio.Blast.NCBIWWW - Code to invoke the NCBI BLAST server over the internet.

DESCRIPTION
    This module provides code to work with the WWW version of BLAST
    provided by the NCBI. https://blast.ncbi.nlm.nih.gov/

FUNCTIONS
    qblast(program, database, sequence, url_base='https://blast.ncbi.nlm.nih.gov/Blast.cgi', auto_format=None, composition_based_statistics=None, db_genetic_code=None, endpoints=None, entrez_query='(none)', expect=10.0, filter=None, gapcosts=None, genetic_code=None, hitlist_size=50, i_thresh=None, layout=None, lcase_mask=None, matrix_name=None, nucl_penalty=None, nucl_reward=None, other_advanced=None, perc_ident=None, phi_pattern=None, query_file=None, query_believe_defline=None, query_from=None, query_to=None, searchsp_eff=None, service=None, threshold=None, ungapped_alignment=None, word_size=None, alignments=500, alignment_view=None, descriptions=500, entrez_links_new_window=None, expect_low=None, expect_h

No exemplo abaixo vamos rodar o blastn no banco de dados de nucleotídeos e vamos usar como query a sequência do NCBI  [GI|8332116](https://www.ncbi.nlm.nih.gov/nuccore/237757283). Em seguida vamos salvar esse resultado em um arquivo no formato XML. Salvar o resultado sempre é uma boa prática, pois o Blast pode demorar muito para rodar e você não quer arriscar perder esses resultados. 

In [5]:
result_handle = NCBIWWW.qblast(program="blastn", database="nt", sequence='237757283', format_type='XML', entrez_query="human[Organism]", ncbi_gi=True)

In [6]:
with open("my_blast.xml", "w") as out_handle:
    out_handle.write(result_handle.read())
result_handle.close()

Agora podemos abrir o resultado do blast com a classe SearchIO e dar uma olhada nos resultados. Esta classe foi especialmente desenvolvida para fazer o parser da saída de diversos programas de alinhamento como Blat, Exonarate e HMMer. O modelo dela consiste de uma hierarquia de objetos do Python.

- QueryResult, este é o primeiro nível que consiste em uma única sequência que foi a query;
- Hit, cada query pode ter zero ou multiplos hits, sendo que cada hit representa um único hit no banco de dados;
- HSP (abreviação para high-scoring pair), eles representam regiões com alinhamento significante entre a query e a sequência do bando de dados. Cada hit pode ter 1 ou mais HSPs;
- HSPFragment, representam alinhamentos continuos entre a query e a sequência do banco de dados. Nos resultados do blast os HSPFragments são unificados, então cada HSP só vai ter um HSPFragment. Contudo, outras ferramentas como Exonerate e BLAT produzem HSP com multiplos HSPFragment.

In [39]:
#listar as queries
from Bio import SearchIO
query = SearchIO.read("my_blast.xml", "blast-xml")

print(f"{query}")

Program: blastn (2.9.0+)
  Query: NM_007294.3 (7224)
         Homo sapiens BRCA1 DNA repair associated (BRCA1), transcript varian...
 Target: nt
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      1  gi|237757283|ref|NM_007294.3|  Homo sapiens BRCA1 DNA r...
            1      1  gi|1677500831|ref|NM_007297.4|  Homo sapiens BRCA1 DNA ...
            2      2  gi|1700447663|ref|NR_027676.2|  Homo sapiens BRCA1 DNA ...
            3      1  gi|555931|gb|U14680.1|HSU14680  Homo sapiens breast and...
            4      1  gi|146147543|gb|BC115036.1|  Homo sapiens cDNA clone IM...
            5      1  gi|1483525317|gb|MG494370.1|  Homo sapiens isolate OV-0...
            6      1  gi|1483525303|gb|MG494363.1|  Homo sapiens isolate OV-0...
            7      1  gi|1483525295|gb|MG494359.1|  Homo sapiens isolate FP-0...
      

In [40]:
#listar os hits
for hit in query:
    print(f"{hit}")

Query: NM_007294.3
       Homo sapiens BRCA1 DNA repair associated (BRCA1), transcript variant ...
  Hit: gi|237757283|ref|NM_007294.3| (7224)
       Homo sapiens BRCA1 DNA repair associated (BRCA1), transcript variant ...
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0         0   13028.80    7224         [0:7224]               [0:7224]
Query: NM_007294.3
       Homo sapiens BRCA1 DNA repair associated (BRCA1), transcript variant ...
  Hit: gi|1677500831|ref|NM_007297.4| (7028)
       Homo sapiens BRCA1 DNA repair associated (BRCA1), transcript variant ...
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  --------

In [41]:
#listar os hsp
for hit in query:
    for hsp in hit:
        print(f"{hsp}")

      Query: NM_007294.3 Homo sapiens BRCA1 DNA repair associated (BRCA1), tr...
        Hit: gi|237757283|ref|NM_007294.3| Homo sapiens BRCA1 DNA repair asso...
Query range: [0:7224] (1)
  Hit range: [0:7224] (1)
Quick stats: evalue 0; bitscore 13028.80
  Fragments: 1 (7224 columns)
     Query - GTACCTTGATTTCGTATTCTGAGAGGCTGCTGCTTAGCGGTAGCCCCTTGGTTTCCGTG~~~AAAAA
             |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||~~~|||||
       Hit - GTACCTTGATTTCGTATTCTGAGAGGCTGCTGCTTAGCGGTAGCCCCTTGGTTTCCGTG~~~AAAAA
      Query: NM_007294.3 Homo sapiens BRCA1 DNA repair associated (BRCA1), tr...
        Hit: gi|1677500831|ref|NM_007297.4| Homo sapiens BRCA1 DNA repair ass...
Query range: [119:7207] (1)
  Hit range: [0:7028] (1)
Quick stats: evalue 0; bitscore 12553.60
  Fragments: 1 (7088 columns)
     Query - GCTGAGACTTCCTGGACGGGGGACAGGCTGTGGGGTTTCTCAGATAACTGGGCCCCTGC~~~TTCCA
             |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||~~~|||||
       Hit - GCTGAGACTT

      Query: NM_007294.3 Homo sapiens BRCA1 DNA repair associated (BRCA1), tr...
        Hit: gi|75875068|gb|DQ190456.1| Homo sapiens clone mck578_U neighbor ...
Query range: [6646:6696] (1)
  Hit range: [126774:126826] (1)
Quick stats: evalue 2.1e-06; bitscore 65.31
  Fragments: 1 (52 columns)
     Query - CTCCAGCCTGGGTGACAGTG--AGACTGTGGCTCAAAAAAAAAAAAAAAAAA
             |||||||||||||||||| |  ||||| || || ||||||||||||||||||
       Hit - CTCCAGCCTGGGTGACAGAGCAAGACTCTGTCTAAAAAAAAAAAAAAAAAAA
      Query: NM_007294.3 Homo sapiens BRCA1 DNA repair associated (BRCA1), tr...
        Hit: gi|75875068|gb|DQ190456.1| Homo sapiens clone mck578_U neighbor ...
Query range: [6403:6475] (1)
  Hit range: [93302:93374] (1)
Quick stats: evalue 7.5e-06; bitscore 63.50
  Fragments: 1 (72 columns)
     Query - GGCATGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCAAGGTGGGCAGATCAC~~~AGGAG
             |||||||||||   ||| |||| ||||||| | |||||||||  |||||||  ||||||~~~|||||
       Hit - GGCATGGTGGCGTGCGCATGTAGTCCCAGCTCCTTGG

       Hit - GCATGGTGGTGCACGCCTGTAATCCCAGCTACTTGGGAGGCTGAGGCAGGAGAATCACTTGA
      Query: NM_007294.3 Homo sapiens BRCA1 DNA repair associated (BRCA1), tr...
        Hit: gi|75874870|gb|DQ190454.1| Homo sapiens clone mck43_A neighbor o...
Query range: [6402:6466] (1)
  Hit range: [47173:47237] (1)
Quick stats: evalue 0.0039; bitscore 53.58
  Fragments: 1 (64 columns)
     Query - GGGCATGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCAAGGTGGGCAGATCACTGGA
             |||||||||||| || |||||||||||||||   |  ||||||  |||| ||   |||||| ||
       Hit - GGGCATGGTGGCACATGCCTGTAATCCCAGCTACTCAGGAGGCTGAGGTAGGAGAATCACTTGA
      Query: NM_007294.3 Homo sapiens BRCA1 DNA repair associated (BRCA1), tr...
        Hit: gi|75874870|gb|DQ190454.1| Homo sapiens clone mck43_A neighbor o...
Query range: [6400:6466] (1)
  Hit range: [7137:7203] (1)
Quick stats: evalue 0.014; bitscore 52.68
  Fragments: 1 (66 columns)
     Query - CTGGGCATGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCAAGGTGGGCAGATCACTGGA
             ||||||||||||||

       Hit - TGGCTCACACCTGTAACCCCAGCACTTTGGGAGGCTGAGGTGAGAGGACTGCTTGAGTC~~~AACAG
      Query: NM_007294.3 Homo sapiens BRCA1 DNA repair associated (BRCA1), tr...
        Hit: gi|75874616|gb|DQ190451.1| Homo sapiens clone mck47_A neighbor o...
Query range: [6469:6696] (1)
  Hit range: [145629:145860] (1)
Quick stats: evalue 2.6e-43; bitscore 187.03
  Fragments: 1 (231 columns)
     Query - CAGGAGTTCGAAACCAGCCTGGCCAACATGGTGAAACCCCATCTCTACTAAAAATACAG~~~AAAAA
             |||||||| || |||||||||| ||||||||  ||||||  ||||||| ||||||||| ~~~|||||
       Hit - CAGGAGTTTGAGACCAGCCTGGGCAACATGGCAAAACCCTGTCTCTACCAAAAATACAA~~~AAAAA
      Query: NM_007294.3 Homo sapiens BRCA1 DNA repair associated (BRCA1), tr...
        Hit: gi|75874616|gb|DQ190451.1| Homo sapiens clone mck47_A neighbor o...
Query range: [6482:6703] (1)
  Hit range: [161335:161564] (1)
Quick stats: evalue 2.6e-43; bitscore 187.03
  Fragments: 1 (229 columns)
     Query - CCAGCCTGGCCAACATG-GTGAAACCCCATCTCTACTAAAAATACAG-----AAATTAG~~~GAAAA

       Hit - GGCATGGTGGCTCACTCCTGTAATCCCAGTACTTTGGGAGGCCAAGG
      Query: NM_007294.3 Homo sapiens BRCA1 DNA repair associated (BRCA1), tr...
        Hit: gi|30039658|gb|AY273801.1| Homo sapiens breast cancer 1, early o...
Query range: [5376:5425] (1)
  Hit range: [62995:63044] (1)
Quick stats: evalue 1.2e-09; bitscore 76.13
  Fragments: 1 (49 columns)
     Query - CTATTTCTGGGTGACCCAGTCTATTAAAGAAAGAAAAATGCTGAATGAG
             || |||  |||||||||||||||||||||||||||||||||||||||||
       Hit - CTCTTTAGGGGTGACCCAGTCTATTAAAGAAAGAAAAATGCTGAATGAG
      Query: NM_007294.3 Homo sapiens BRCA1 DNA repair associated (BRCA1), tr...
        Hit: gi|30039658|gb|AY273801.1| Homo sapiens breast cancer 1, early o...
Query range: [6506:6695] (1)
  Hit range: [13694:13882] (-1)
Quick stats: evalue 1.4e-08; bitscore 72.52
  Fragments: 1 (194 columns)
     Query - CCCATCTCTACTAAAAATACAGAAATTAGCCGGTCATGGTGGTGGACACCTGTAATCCC~~~AAAAA
             ||||||||||| | |||   | |||||||| || ||   ||      ||||||| ||| ~~~||||

In [42]:
#número de hits do resultado 
print(f"Número de Hits = {len(query)}\n{query}")

Número de Hits = 50
Program: blastn (2.9.0+)
  Query: NM_007294.3 (7224)
         Homo sapiens BRCA1 DNA repair associated (BRCA1), transcript varian...
 Target: nt
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      1  gi|237757283|ref|NM_007294.3|  Homo sapiens BRCA1 DNA r...
            1      1  gi|1677500831|ref|NM_007297.4|  Homo sapiens BRCA1 DNA ...
            2      2  gi|1700447663|ref|NR_027676.2|  Homo sapiens BRCA1 DNA ...
            3      1  gi|555931|gb|U14680.1|HSU14680  Homo sapiens breast and...
            4      1  gi|146147543|gb|BC115036.1|  Homo sapiens cDNA clone IM...
            5      1  gi|1483525317|gb|MG494370.1|  Homo sapiens isolate OV-0...
            6      1  gi|1483525303|gb|MG494363.1|  Homo sapiens isolate OV-0...
            7      1  gi|1483525295|gb|MG494359.1|  Homo sapiens is

In [43]:
#Acessar o primeiro hit do resultado
print(f"{query[0]}")

Query: NM_007294.3
       Homo sapiens BRCA1 DNA repair associated (BRCA1), transcript variant ...
  Hit: gi|237757283|ref|NM_007294.3| (7224)
       Homo sapiens BRCA1 DNA repair associated (BRCA1), transcript variant ...
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0         0   13028.80    7224         [0:7224]               [0:7224]


In [44]:
#acessar o último resultado
print(f"{query[-1]}")

Query: NM_007294.3
       Homo sapiens BRCA1 DNA repair associated (BRCA1), transcript variant ...
  Hit: gi|29126449|gb|AC060780.18| (110669)
       Homo sapiens chromosome 17, clone RP11-242D8, complete sequence
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0         0    6180.53    3429       [901:4330]            [5861:9290]
          1    5e-103     385.40     213          [0:213]          [39758:39971]
          2   1.5e-90     344.83     296      [6403:6697]          [21560:21854]
          3   1.5e-90     343.93     306      [6400:6703]          [48114:48420]
          4   2.3e-88     337.62     297      [6402:6696]          [17685:17981]
          5   9.7e-87     332.20     291      [6409:6696]        [106511:106801]
          6   1.4e-84     324.99     296      [6400:6694]

In [45]:
#Slice dos resultados
blast_slice = query[:3]
print(f"{blast_slice}")

Program: blastn (2.9.0+)
  Query: NM_007294.3 (7224)
         Homo sapiens BRCA1 DNA repair associated (BRCA1), transcript varian...
 Target: nt
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      1  gi|237757283|ref|NM_007294.3|  Homo sapiens BRCA1 DNA r...
            1      1  gi|1677500831|ref|NM_007297.4|  Homo sapiens BRCA1 DNA ...
            2      2  gi|1700447663|ref|NR_027676.2|  Homo sapiens BRCA1 DNA ...


Assim como em um dicionário em Python você pode consultar acessa-lo diretamente através do ID do Hit.

In [18]:
query["gi|1700447663|ref|NR_027676.2|"]

Hit(id='gi|1700447663|ref|NR_027676.2|', query_id='NM_007294.3', 2 hsps)

Da mesma foram também é possível consultar se determinado Hit existe.

In [21]:
"gi|1700447663|ref|NR_027676.2|" in query

True

In [23]:
"gi|1700447663|ref|NR_027676" in query

False

Também é possívem retornar a posição em que determinado Hit se encontra.

In [24]:
query.index("gi|1700447663|ref|NR_027676.2|")

2

### 5.1 Atributos dos Hits e HSP
Vamos ver como listar alguns atributos dos Hits e HSPs

In [26]:
#Aqui pegamos um dos Hits da query
blast_hit = query[45]
blast_hit

Hit(id='gi|75874526|gb|DQ190450.1|', query_id='NM_007294.3', 296 hsps)

In [48]:
#atributos do Hit
print(f"Accession: {blast_hit.accession}")
print(f"Blast_id: {blast_hit.blast_id}")
print(f"Description: {blast_hit.description}")

Accession: DQ190450
Blast_id: gi|75874526|gb|DQ190450.1|
Description: Homo sapiens clone mck432_A neighbor of BRCA1 gene 1 (NBR1) gene, partial cds; and hypothetical protein LOC10230 (NBR2) and breast cancer 1 early onset (BRCA1) genes, complete cds


In [49]:
#Aqui pegamos um dos HSPs
blast_hsp = blast_hit.hsps[0]
blast_hsp

HSP(hit_id='gi|75874526|gb|DQ190450.1|', query_id='NM_007294.3', 1 fragments)

In [50]:
blast_hsp.evalue

0.0

In [51]:
blast_hsp.hit_start

99645

In [52]:
blast_hsp.query_span

3429

In [53]:
blast_hsp.aln_span

3429

In [54]:
blast_hsp.gap_num

0

In [55]:
blast_hsp.ident_num

3429

In [56]:
blast_hsp.query

SeqRecord(seq=Seq('GCTGCTTGTGAATTTTCTGAGACGGATGTAACAAATACTGAACATCATCAACCC...GGT', DNAAlphabet()), id='NM_007294.3', name='aligned query sequence', description='Homo sapiens BRCA1 DNA repair associated (BRCA1), transcript variant 1, mRNA', dbxrefs=[])

In [57]:
blast_hsp.hit

SeqRecord(seq=Seq('GCTGCTTGTGAATTTTCTGAGACGGATGTAACAAATACTGAACATCATCAACCC...GGT', DNAAlphabet()), id='gi|75874526|gb|DQ190450.1|', name='aligned hit sequence', description='Homo sapiens clone mck432_A neighbor of BRCA1 gene 1 (NBR1) gene, partial cds; and hypothetical protein LOC10230 (NBR2) and breast cancer 1 early onset (BRCA1) genes, complete cds', dbxrefs=[])

### Exercício 7

Dado um arquivo com uma sequência no formato FASTA que já se encontra no diretório atual com o nome 'exercicio.fa':
- Abra-o utilizando o SeqIO.read e faça um blastn contra no banco de dados nt, sem limitar a um organismo; 
- Pegue o primeiro Hit, liste os HSPs e ache uma inserção na query que não tem no banco de dados;
- Com as coordenadas da inserção selecione apenas essa região na sua query e salve em um arquivo com o nome "dna_message.dna". 
- Após salvar execute a célula com os seguintes comandos e desvende a menssagem:

%%bash

python2.7 /srv/conda/envs/notebook/lib/python3.7/site-packages/dna/dna.py -d dna_message.dna

cat dna_message.decoded

In [67]:
from Bio import SeqIO, SearchIO
from Bio.Blast import NCBIWWW

record = #Acrescente o código para ler o arquivo fasta usando o SeqIO.read

result_handle = NCBIWWW.qblast(program="blastn", database="nt", sequence=record.seq, format_type='XML')

with open("my_blast.xml", "w") as out_handle:
    out_handle.write(result_handle.read())

In [75]:
query = SearchIO.read("my_blast.xml", "blast-xml")

#Acrescente abaixo o código para listar o primeiro hit da query e descubra a região de inserção


In [70]:
with open('dna_message.dna', 'w') as out_dna:
    insercao = #acrescente o código para pegar apenas a sequência apenas 
    out_dna.write(str(insercao))

Seq('TCAGTCGCATCGCAGCGTAGTGCTGATAGTGTCATATGATATCTACATCACAGT...CAG', SingleLetterAlphabet())

In [76]:
%%bash

python2.7 /srv/conda/envs/notebook/lib/python3.7/site-packages/dna/dna.py -d dna_message.dna

cat dna_message.decoded

Couldn't find program: 'bash'
